# Installing Environment

In [ ]:
!pip install kaggle-environments>=0.1.6

# Importing Dependencies

- kaggle environment
- random
- numpy
- os
- inspect

In [ ]:
from kaggle_environments import make, evaluate, utils, agent
import random
import numpy as np
import os
import inspect

# Instantiating Game environment

making game environment for *connectx* using *make function* of kaggle environment 

In [ ]:
env=make("connectx",debug=True)
env.render()

# Helper Functions

- drop_piece: return grid status after player drops a piece
- check_winning_move : used to check if dropping a piece in a column of board leads to a winning move or not

In [ ]:
# Gets board at next step if agent drops piece in selected column
def drop_piece(grid, col, piece, config):
    next_grid = grid.copy()
    for row in range(config.rows-1, -1, -1):
        if next_grid[row][col] == 0:
            break
    next_grid[row][col] = piece
    return next_grid

# Returns True if dropping piece in column results in game win
def check_winning_move(obs, config, col, piece):
    # Convert the board to a 2D grid
    grid = np.asarray(obs.board).reshape(config.rows, config.columns)
    next_grid = drop_piece(grid, col, piece, config)
    # horizontal
    for row in range(config.rows):
        for col in range(config.columns-(config.inarow-1)):
            window = list(next_grid[row,col:col+config.inarow])
            if window.count(piece) == config.inarow:
                return True
    # vertical
    for row in range(config.rows-(config.inarow-1)):
        for col in range(config.columns):
            window = list(next_grid[row:row+config.inarow,col])
            if window.count(piece) == config.inarow:
                return True
    # positive diagonal
    for row in range(config.rows-(config.inarow-1)):
        for col in range(config.columns-(config.inarow-1)):
            window = list(next_grid[range(row, row+config.inarow), range(col, col+config.inarow)])
            if window.count(piece) == config.inarow:
                return True
    # negative diagonal
    for row in range(config.inarow-1, config.rows):
        for col in range(config.columns-(config.inarow-1)):
            window = list(next_grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
            if window.count(piece) == config.inarow:
                return True
    return False

# Creating an agent

we create an agent in form of a function that takes two parameters:

1. observation: has recent board information like current board configuration after move.
2. configuration: which has board configuration like number of columns, rows, number of rows to match

In this function an agent is created that works as follow:

- if winning move then return that column that leads to winning move
- blocks winning move of opponent
- if above two rules do not follow then drop piece at random column

In [ ]:
def my_agent(obs, config):
    opponent_piece = 1 if obs.mark == 2 else 2
    choice = []
    for col in range(config.columns):
        if check_winning_move(obs,config,col,obs.mark):
            return col
        elif check_winning_move(obs,config,col,opponent_piece):
            choice.append(col)
    if len(choice):
        return random.choice(choice)
    valid_moves = [col for col in range(config.columns) if obs.board[col] == 0]
    return random.choice(valid_moves)

# Evaluating agent

### evaluation against agent that drops piece at random as player 1

In [ ]:
env.reset()
env.run([my_agent,"random"])
env.render(mode="ipython",width=500,height=500)

### evaluation against agent that drops piece at random as player 2

In [ ]:
env.reset()
env.run(["random",my_agent])
env.render(mode="ipython",width=500,height=500)

### evaluation against agent that drops piece using negamax as player 1

In [ ]:
env.reset()
env.run([my_agent,"negamax"])
env.render(mode="ipython",width=500,height=500)

### evaluation against agent that drops piece using negamax as player 2

In [ ]:
env.reset()
env.run(["negamax",my_agent])
env.render(mode="ipython",width=500,height=500)

# Calculating Win percentage

- against random and negamax agent

*evaluate function* is used to get rewards which are of form list of list ([[1,-1],[1,-1]-----]]) 1 denote *win* while -1 denote *loose* and None denote *no-result*

Here, first we find winning record when player 1 is our agent and then find winning record when our agent is playing as player 2. This ensure that plays are fair 

In [ ]:
def win_percentage(player,opponent,num_episodes=10):
    episodes = num_episodes//2
    outcomes = evaluate("connectx",[player,opponent],num_episodes=episodes)
    outcomes += [[b,a] for [a,b] in evaluate("connectx",[opponent,player],num_episodes=num_episodes-episodes)]
    wins = outcomes.count([1,-1])
    losses = outcomes.count([-1,1])
    return (np.sum(wins) / len(outcomes))*100

In [ ]:
random_mean_reward = win_percentage(my_agent,"random",num_episodes=10)
negamax_mean_reward = win_percentage(my_agent,"negamax",num_episodes=10)

In [ ]:
print("My Agent V/S Random Agent ", random_mean_reward,"%")
print("My Agent V/S Negamax Agent ",negamax_mean_reward,"%")

# Play against the agent

In [ ]:
env.play([my_agent,None],width=500,height=500)

# Creating Submission file

In [ ]:
submission_file="submission.py"

remove previously created submission file

In [ ]:
if os.path.exists(submission_file):
    os.remove(submission_file)

### Helper Functions for writing code to a file

- write_agent_dependencies: write all dependencies to python file
- write_function_to_file: write function code to file

In [ ]:
def write_agent_dependencies(file,dependencies):
    with open(file,"a" if os.path.exists(file) else "w") as f:
        for dependency in dependencies:
            f.write(f"import {dependency}\n")
        print(f"depedencies written to {file}")
    
def write_function_to_file(file,function):
    with open(file, "a" if os.path.exists(file) else "w") as f:
        f.write("")
        f.write(inspect.getsource(function))
        print(f"function written to {file}")

define required dependencies here

In [ ]:
dependencies=["numpy as np","random"]

`Note: submission file should be a python file with the last 'def' accepting an observation and returning an action`

To create the submission, an agent function should be fully encapsulated (no external dependencies).

When agent is being evaluated against others, it will not have access to the Kaggle docker image. Only the following can be imported: Python Standard Library Modules, gym, numpy, scipy, pytorch (1.3.1, cpu only), and more may be added later.

writing `submission.py` file

- write all dependencies
- write all function dependencies to file
- write agent function to file

In [ ]:
write_agent_dependencies("submission.py",dependencies)
write_function_to_file("submission.py",drop_piece)
write_function_to_file("submission.py",check_winning_move)
write_function_to_file("submission.py",my_agent)

In [ ]:
with open("submission.py","r") as f:
    print(f.read())

# Validating submission file

This is to check if submission file is working properly or not ie.. agent function is encapsulated with all dependencies function are included or not.
It we submit a non encapsulated agent file we will get *validation error* 

In [ ]:
import sys
out = sys.stdout
submission = utils.read_file("/kaggle/working/submission.py")
sys.stdout = out
saved_agent = agent.get_last_callable(submission)
env = make("connectx", debug=True)
env.run([saved_agent, saved_agent])
print("Success!" if env.state[0].status == env.state[1].status == "DONE" else "Failed...")